# Image Cation project Using CNN amd LSTM:

In [9]:
!pip install tensorflow numpy pandas matplotlib pillow nltk tqdm # install the dependencies


In [10]:
!pip install kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [11]:
!kaggle datasets download -d adityajn105/flickr8k
!unzip flickr8k.zip -d /content/


Dataset URL: https://www.kaggle.com/datasets/adityajn105/flickr8k
License(s): CC0-1.0
flickr8k.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  flickr8k.zip
replace /content/Images/1000268201_693b08cb0e.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [12]:
!unzip flickr8k.zip -d /content/flickr8k/



Archive:  flickr8k.zip
replace /content/flickr8k/Images/1000268201_693b08cb0e.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import re
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import nltk
from nltk.translate.bleu_score import sentence_bleu
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
image_path = "/content/drive/MyDrive/archive (13)/Images"


In [5]:
captions_file = "/content/drive/MyDrive/archive (13)/captions.txt"


In [6]:
df = pd.read_csv(captions_file)
df.head()

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


Pre-processing of the caption.

In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z ]", "", text)
    return text

df["caption"] = df["caption"].apply(clean_text)
df["caption"] = df["caption"].apply(lambda x: "startseq " + x + " endseq")


In [13]:
#Convert all of them to lowercase
#df["caption"] = df["caption"].apply(lambda x:x.lower())

In [14]:
# Apply cleaning function to all captions
#df["caption"] = df["caption"].apply(lambda x:re.sub("[^A-Za-z0-9]", " ", x))

In [17]:
 # Add start & end tokens
#df["caption"] = df["caption"].apply(lambda x: '<start> ' + x + ' <end>')

In [8]:

# Load pre-trained InceptionV3
from tensorflow.keras.models import Model
inception = InceptionV3(weights='imagenet')
model_new = Model(inception.input, inception.layers[-2].output)

In [9]:
from tqdm import tqdm
from PIL import Image
import numpy as np
from PIL import Image
def preprocess_image(image_path):
    img = Image.open(image_path).resize((299, 299))  # Resize image to (299, 299)
    img = np.array(img) / 255.0  # Normalize the pixel values to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

In [ ]:

image_features = {}
for img_name in tqdm(os.listdir(image_path)):
    img_path = os.path.join(image_path, img_name)
    img = preprocess_image(img_path)
    feature = model_new.predict(img)
    image_features[img_name] = feature

  0%|          | 0/8101 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


  0%|          | 1/8101 [00:06<14:51:17,  6.60s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step


  0%|          | 2/8101 [00:06<6:33:25,  2.91s/it] 

...

 55%|█████▍    | 4438/8101 [38:46<42:04,  1.45it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step


 55%|█████▍    | 4439/8101 [38:47<43:38,  1.40it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step


 55%|█████▍    | 4440/8101 [38:47<45:01,  1.36it/s]

In [9]:
pickle.dump(image_features, open("/content/image_features.pkl", "wb"))


In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["caption"])

vocab_size = len(tokenizer.word_index) + 1
# Save tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

max_length = max(len(x.split()) for x in df["caption"])
# Save tokenizer

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df["caption"])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')


In [12]:

embedding_dim = 256
lstm_units = 256

# Image feature input
image_input = Input(shape=(2048,))
image_layer = Dense(embedding_dim, activation='relu')(image_input)

# Text input
text_input = Input(shape=(max_length,))
text_layer = Embedding(vocab_size, embedding_dim, mask_zero=True)(text_input)
text_layer = LSTM(lstm_units, return_sequences=False)(text_layer)

# Merge
decoder = tf.keras.layers.add([image_layer, text_layer])
decoder = Dense(vocab_size, activation='softmax')(decoder)

# Compile Model
model = Model(inputs=[image_input, text_input], outputs=decoder)
model.compile(loss='categorical_crossentropy', optimizer=Adam())

model.summary()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 37)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 2048)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 37, 256)        │      2,247,936 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, 37)             │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 256)            │        524,544 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 256)            │        525,312 │ embedding[0][0],       │
│                           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 256)            │              0 │ dense[0][0],           │
│                           │                        │                │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 8781)           │      2,256,717 │ add[0][0]              │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,554,509 (21.19 MB)

 Trainable params: 5,554,509 (21.19 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Prepare training data
X_image, X_text, y = [], [], []

for i in range(len(df)):
    img_id = df["image"][i]
    img_feature = image_features[img_id].reshape(2048,)
    seq = sequences[i]

    for j in range(1, len(seq)):
        in_seq, out_seq = seq[:j], seq[j]
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

        X_image.append(img_feature)
        X_text.append(in_seq)
        y.append(out_seq)

# Convert to numpy arrays
X_image = np.array(X_image)
X_text = np.array(X_text)
y = np.array(y)

# Train model
model.fit([X_image, X_text], y, batch_size=64, epochs=10)
# Save model
model.save("image_captioning_model.h5")


In [ ]:
# Load saved model & tokenizer
model = load_model("image_captioning_model.h5")

with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

with open("image_features.pkl", "rb") as f:
    image_features = pickle.load(f)


In [ ]:
def generate_caption(image_path):
    img = preprocess_image(image_path)
    feature = model_new.predict(img).reshape(1, 2048)

    caption = "startseq"
    for _ in range(max_length):
        seq = tokenizer.texts_to_sequences([caption])[0]
        seq = pad_sequences([seq], maxlen=max_length)

        pred = model.predict([feature, seq])
        word = tokenizer.index_word[np.argmax(pred)]

        if word == "endseq":
            break
        caption += " " + word

    return caption.replace("startseq", "").replace("endseq", "").strip()

# Test Caption Generation
test_image = "/content/flickr8k/Images/example.jpg"
plt.imshow(Image.open(test_image))
plt.axis("off")
plt.show()
print("Generated Caption:", generate_caption(test_image))
